In [8]:
import os
import math
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

from sklearn.model_selection import train_test_split
from keras.models import Sequential, model_from_json
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, ZeroPadding2D, Convolution2D, Dropout
from keras.preprocessing.image import ImageDataGenerator, img_to_array
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from sklearn.metrics import f1_score
from PIL import Image

from constant_values import *
from img_preprocessing import *
from LIDAR_model import *
from im_postprocess import *
from predictions import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load full image dataset and augmentation done

In [2]:
tr_imgs = data_augmentation("../Data/training/images/")      #100 images --> 2400 images
gt_imgs = data_augmentation("../Data/training/groundtruth/") #100 images --> 2400 images

Image 100/100 is being processed ... Shuffle data ...
Image 100/100 is being processed ... Shuffle data ...


In [9]:
tr_imgs2 = tr_imgs[:TRAINING_SIZE, :, :, :]
gt_imgs2 = gt_imgs[:TRAINING_SIZE, :, :, :]

In [10]:
# Extract each images into patches (numpy arrays).
img = extract_data(tr_imgs2)  #shape: ((400/16)^2 * TRAINING_SIZE, 16, 16, 3)
gt = extract_labels(gt_imgs2) #shape: ((400/16)^2 * TRAINING_SIZE, 2)

In [11]:
#Test model simple

# Create model
# https://keras.io/models/sequential/
#mod = Sequential()

# Add a layer
# https://keras.io/layers/about-keras-layers/
# https://keras.io/activations/
#mod.add(Conv2D(64, kernel_size=IMG_PATCH_SIZE, activation='relu',input_shape=(IMG_PATCH_SIZE, IMG_PATCH_SIZE, NUM_CHANNELS)))
#mod.add(Flatten())
#mod.add(Dense(2, activation='softmax'))
        
        
# https://keras.io/optimizers/
# https://keras.io/losses/
#mod.compile(optimizer='sgd', loss='mean_squared_error', metrics=['accuracy']) #f1 = 0.43
#mod.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy']) #f1 = 0.43

In [12]:
model = model_Lidar((16, 16, 3))
model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




In [ ]:
TEST_SIZE = 0.33 # To be studied

# Step 0: Shuffle samples
np.random.seed(0)
np.random.shuffle(img)
np.random.seed(0)
np.random.shuffle(gt)

# Step 1: Split into validation and training set     
Xtrain, Xtest, gt_train, gt_test = train_test_split(img, gt, test_size=TEST_SIZE, random_state=1)

# Step 2: Give weights to classes ?
c_weight = {1: 3., 
            0: 1.}

# Step 3: Generate Generators
train_datagenerator = ImageDataGenerator(horizontal_flip=True, vertical_flip=True)
test_datagenerator  = ImageDataGenerator()

train_generator = train_datagenerator.flow(Xtrain, gt_train, batch_size=BATCH_SIZE)
test_generator  = test_datagenerator.flow(Xtest, gt_test, batch_size=BATCH_SIZE)

# Step 4: Early stop
early_stop_callback = EarlyStopping(monitor='val_acc', min_delta=0, patience=5, verbose=0, mode='max', restore_best_weights=True)

# Step 5: Training
model.fit_generator(train_generator,
            validation_data=test_generator,
            steps_per_epoch=math.ceil(TRAINING_SIZE / BATCH_SIZE), #len(train_generator)/16,
            epochs=NUM_EPOCHS,
            callbacks = [early_stop_callback],
            class_weight=c_weight,
            validation_steps=math.ceil(len(test_generator)/BATCH_SIZE))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/100





13/13 [==============================] - 467s 36s/step - loss: 1.0072 - acc: 0.7308 - val_loss: 0.6669 - val_acc: 0.7994
Epoch 2/100
13/13 [==============================] - 410s 32s/step - loss: 0.9787 - acc: 0.7933 - val_loss: 0.6324 - val_acc: 0.7951
Epoch 3/100
13/13 [==============================] - 345s 27s/step - loss: 0.9097 - acc: 0.8317 - val_loss: 0.6249 - val_acc: 0.8017
Epoch 4/100
13/13 [==============================] - 461s 35s/step - loss: 0.9433 - acc: 0.8077 - val_loss: 0.6162 - val_acc: 0.7897
Epoch 5/100
13/13 [==============================] - 375s 29s/step - loss: 0.9295 - acc: 0.8221 - val_loss: 0.6266 - val_acc: 0.8067
Epoch 6/100
12/13 [==========================>...] - ETA: 5s - loss: 0.9868 - acc: 0.7865 

In [48]:
train_data_filename = "../Data/training/images/"

prediction_training_dir = "predictions_training_lidar_model_LEO/"
if not os.path.isdir(prediction_training_dir):
    os.mkdir(prediction_training_dir)

for i in range(1, TRAINING_SIZE+1):
    pimg = get_prediction_with_groundtruth(img[i], model)
    Image.fromarray(pimg).save(prediction_training_dir + "prediction_" + str(i) + ".png")
    oimg = get_prediction_with_overlay(img[i], model)
    oimg.save(prediction_training_dir + "overlay_" + str(i) + ".png")
    sys.stdout.write("\rImage {}/{} is being processed".format(i,TRAINING_SIZE+1))
    sys.stdout.flush()

Image 1/51 is being processed

/Users/leobouraux/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:139: RuntimeWarning: invalid value encountered in true_divide


Image 50/51 is being processed

In [50]:

# serialize model to JSON
OUTPUT_FILENAME = "simple_model"
model_json = model.to_json()
print("1")
with open(OUTPUT_FILENAME + ".json", "w") as json_file:
    json_file.write(model_json)
    
print("2")
# serialize weights to HDF5
model.save_weights(OUTPUT_FILENAME + ".h5")
print("Saved model to disk")

validation_data_prediction = model.predict_classes(Xtest, verbose = 1)
print("3")
validation_label = []
for e in gt_test:
    print("4")
    if (e[0] == 0):
        validation_label.append(1)
    else:
        validation_label.append(0)
        
validation_label = np.array(validation_label)     
print("F1 score = "+str(f1_score(validation_data_prediction, validation_label, labels=['1'])))

1
2
Saved model to disk
10313/10313 [==============================] - 1177s 114ms/step
3


NameError: name 'label_te' is not defined